NumPy attempt

In [1]:
import numpy as np
from ncon import ncon
import matplotlib.pyplot as plt

In [2]:
def rnd(*x):
    return np.random.randn(*x) + 1j*np.random.randn(*x)

def overlap(nodes): # needs revising

    mps = []
    rank = len(nodes)
    indices = []
    dummy = 1
    for i in range(rank):
        if i == op_site:
            mps += [nodes[op_site], o, nodes[op_site].conj()]
            indices += [dummy, dummy+2, dummy+4], [dummy+2, dummy+3], [dummy+1, dummy+3, dummy+5]
            dummy += 4
            continue
        mps += [nodes[i], nodes[i].conj()]
        if i == 0:
            indices += [dummy, dummy+1], [dummy, dummy+2]
            dummy += 1
            continue
        if i == rank-1:
            indices += [dummy, dummy+2], [dummy+1, dummy+2]
            break
        indices += [dummy, dummy+2, dummy+3], [dummy+1, dummy+2, dummy+4]
        dummy += 3
    
    return ncon(mps, indices)

def left_env(nodes, site):
    """
    Evironment to the left of site specified (sites numbered left to right starting from 1). 
    """
    mps = []
    indices = []
    dummy = 1
    for i in range(site-1):
        mps += [nodes[i], nodes[i].conj()]
        if site == 2:
            indices += [dummy, -1], [dummy, -2]
            break
        elif i == 0:
            indices += [dummy, dummy+1], [dummy, dummy+2]
            dummy += 1
            continue
        elif i == site-2:
            indices += [dummy, dummy+2, -1], [dummy+1, dummy+2, -2]
            break
        indices += [dummy, dummy+2, dummy+3], [dummy+1, dummy+2, dummy+4]
        dummy += 3

    return ncon(mps, indices)

def right_env(nodes, site):
    """
    Evironment to the left of site specified (sites numbered left to right starting from 1).
    """
    rank = len(nodes)
    mps = []
    indices = []
    dummy = 1
    for i in range(site, rank):
        mps += [nodes[i], nodes[i].conj()]
        if site == rank-1:
            indices += [-1, dummy], [-2, dummy]
            break
        if i == site:
            indices += [-1, dummy, dummy+1], [-2, dummy, dummy+2] 
            dummy += 1
            continue
        elif i == rank-1:
            indices += [dummy, dummy+2], [dummy+1, dummy+2]
            break
        indices += [dummy, dummy+2, dummy+3], [dummy+1, dummy+2, dummy+4]
        dummy += 3

    return ncon(mps, indices)

def norm(nodes):

    mps = []
    rank = len(nodes)
    indices = []
    dummy = 1
    for i in range(rank):

        mps += [nodes[i], nodes[i].conj()]
        if i == 0:
            indices += [dummy, dummy+1], [dummy, dummy+2]
            dummy += 1
            continue
        if i == rank-1:
            indices += [dummy, dummy+2], [dummy+1, dummy+2]
            break
        indices += [dummy, dummy+2, dummy+3], [dummy+1, dummy+2, dummy+4]
        dummy += 3

    return ncon(mps, indices)

def retrieve(nodes, bitstring):

    rank = len(nodes)
    indices = []
    dummy = 1
    for i in range(rank):

        if i == 0:
            indices += [[-(i+1), dummy]]
            continue
        if i == rank-1:
            indices += [[dummy, -(i+1)]]
            break
        indices += [[dummy, -(i+1), dummy+1]]
        dummy += 1

    return ncon(nodes, indices)[bitstring]

In [3]:
def canonicalise(d=2, D=9, rank=7, bitstring = (0,1,0,1,0)):
    """ 
    Legend: 
    first:  nodes[site][physical, virtual]
    middle: nodes[site][virtual, physical, virtual]
    last:   nodes[site][virtual, physical] 
    """

    nodes = [rnd(d,D), *[rnd(D,d,D) for i in range(rank-2)], rnd(D,d)] 

    nrm = norm(nodes)
    cmp = retrieve(nodes, bitstring)

    # L -> R
    for i in range(rank-1):
        
        if not i == 0:
            nodes[i] = nodes[i].reshape(nodes[i].shape[0]*nodes[i].shape[1],D)

        # svd
        svd = np.linalg.svd(nodes[i])
        s = np.zeros(nodes[i].shape)
        np.fill_diagonal(s, svd[1])

        if i == 0:
            # contract to form new nodes from svd
            nodes[i] = svd[0]
            nodes[i+1] = np.einsum("ij,jk,klm->ilm", s, svd[2], nodes[i+1])

        elif i == rank-2:
            # contract to form new nodes from svd
            nodes[i] = svd[0].reshape(nodes[i-1].shape[-1],d,-1) 
            nodes[i+1] = np.einsum("ij,jk,kl->il", s, svd[2], nodes[i+1])

        else:
            # contract to form new nodes from svd
            nodes[i] = svd[0].reshape(nodes[i-1].shape[-1],d,-1) 
            nodes[i+1] = np.einsum("ij,jk,klm->ilm", s, svd[2], nodes[i+1])

        # checks
        assert np.isclose(nrm, norm(nodes))
        assert np.allclose(cmp, retrieve(nodes, bitstring))

    # checks
    assert np.allclose(left_env(nodes, 3), np.eye(left_env(nodes, 3).shape[0]))
    assert np.isclose(nrm, norm(nodes))
    assert np.allclose(cmp, retrieve(nodes, bitstring))

    # R -> L
    for i in range(rank-1, 0, -1):

        if i == rank-1:
            # svd the evironment to the right of site i
            R = nodes[i] @ nodes[i].T.conj()

        else:
            # svd the evironment to the right of site i
            R = ncon([nodes[i], nodes[i].conj(), R], [[-1,2,3], [-2,2,4], [3,4]])

        svd = np.linalg.svd(R)
        s = np.zeros(R.shape)
        np.fill_diagonal(s, svd[1])

        # gauge transformations
        if i == rank-1:
            nodes[i] = np.einsum("ij,jk->ik", svd[2], nodes[i])
            nodes[i-1] = np.einsum("ijk,kl->ijl", nodes[i-1], svd[0])

        elif i == 1:
            nodes[i] = np.einsum("ij,jkl->ikl", svd[2], nodes[i])
            nodes[i-1] = np.einsum("ij,jk->ik", nodes[i-1], svd[0])

        else:
            nodes[i] = np.einsum("ij,jkl->ikl", svd[2], nodes[i])
            nodes[i-1] = np.einsum("ijk,kl->ijl", nodes[i-1], svd[0])

        R = svd[2] @ R @ svd[0]

    # checks
    assert np.isclose(nrm, norm(nodes))
    assert np.allclose(cmp, retrieve(nodes, bitstring))

    return nodes

In [4]:
nodes = canonicalise()

Quimb Attempt

In [ ]:
import quimb as qu
import quimb.tensor as qtn

In [ ]:
a1 = qtn.Tensor(qu.rand_ket(4, seed=1).reshape(2, 2), inds=("p0", "b0"), tags="a1")
b1 = qtn.Tensor(qu.rand_ket(4, seed=1).reshape(2, 2), inds=("p0", "b1"), tags="b1")
a2 = qtn.Tensor(qu.rand_ket(8, seed=2).reshape(2, 2, 2), inds=("b0", "p1", "b2"), tags="a2")
b2 = qtn.Tensor(qu.rand_ket(8, seed=2).reshape(2, 2, 2), inds=("b1", "p1", "b3"), tags="b2")
a3 = qtn.Tensor(qu.rand_ket(8, seed=3).reshape(2, 2, 2), inds=("b2", "p2", "b4"), tags="a3")
o = qtn.Tensor(qu.rand_ket(4).reshape(2, 2), inds=("p2", "p3"), tags="o")
b3 = qtn.Tensor(qu.rand_ket(8, seed=3).reshape(2, 2, 2), inds=("b3", "p3", "b5"), tags="b3")
a4 = qtn.Tensor(qu.rand_ket(8, seed=4).reshape(2, 2, 2), inds=("b4", "p4", "b6"), tags="a4")
b4 = qtn.Tensor(qu.rand_ket(8, seed=4).reshape(2, 2, 2), inds=("b5", "p4", "b7"), tags="b4")
a5 = qtn.Tensor(qu.rand_ket(4, seed=5).reshape(2, 2), inds=("b6", "p5"), tags="a5")
b5 = qtn.Tensor(qu.rand_ket(4, seed=5).reshape(2, 2), inds=("b7", "p5"), tags="b5")

TN = a1 & b1 & a2 & b2 & a3 & o & b3 & a4 & b4 & a5 & b5
TN.draw(show_inds="all", font_size_inner=10)

In [ ]:
TN = a1 & a1 & a2 & b2 & a3 & o & b3 & a4 & b4 & a5 & b5
TN.draw(show_inds="all", font_size_inner=10)

In [ ]:
TN = (a1 @ a2) & (b1 @ b2) & a3 & o & b3 & (a4 @ a5) & (b4 @ b5)
TN.draw(show_inds="all", font_size_inner=10)

In [ ]:
TN.reindex({"0": "0", "3": "0", "10": "10", "13": "10"}, inplace=True)
TN.draw(show_inds="all", font_size_inner=10)

In [ ]:
TN[{"a1", "a2"}].data.reshape(4,2)
TN[{"b1", "b2"}].data.reshape(4,2)
TN[{"a4", "a5"}].data.reshape(2,4)
TN[{"b4", "b5"}].data.reshape(2,4)

In [ ]:
TN[{"a1", "a2"}].data.reshape(4,2)

TensorNetwork Attempt

In [ ]:
import tensornetwork as tn
import matplotlib.pyplot as plt

In [ ]:
phys_dim = 2
bond_dim = 2
rank = 5
opindex = rank//2

In [ ]:
# build the mps:
# the state is canonically normalized when we define the class FiniteMPS
mps = tn.FiniteMPS.random(
  d = [phys_dim for _ in range(rank)],
  D = [bond_dim for _ in range(rank-1)],
  dtype = np.float32
  )

mps.tensors[0] = mps.tensors[0].reshape(2,2)
mps.tensors[-1] = mps.tensors[-1].reshape(2,2)

In [ ]:
# connect the edges in the mps and conjugate mps
uppernodes = [tn.Node(tensor, f'A{i}') for i, tensor in enumerate(mps.tensors)]
lowernodes = [tn.Node(tensor.conj(), f'B{i}') for i, tensor in enumerate(mps.tensors)]
opnode = tn.Node(np.random.randn(phys_dim, phys_dim), "o")

upper_edgs = [uppernodes[0][1] ^ uppernodes[1][0]]
lower_edgs = [lowernodes[0][1] ^ lowernodes[1][0]]
phys_edgs = [uppernodes[0][0] ^ lowernodes[0][0]]
for k in range(rank-2):
  upper_edgs.append(uppernodes[k+1].edges[2] ^ uppernodes[k+2].edges[0])
  lower_edgs.append(lowernodes[k+1].edges[2] ^ lowernodes[k+2].edges[0])
for k in range(rank-1):
  if k+1 == opindex:
    phys_edgs.append(uppernodes[opindex][1] ^ opnode[0])
    phys_edgs.append(lowernodes[opindex][1] ^ opnode[1])
    continue
  phys_edgs.append(uppernodes[k+1].edges[1] ^ lowernodes[k+1].edges[1])

In [ ]:
phys_edgs

In [ ]:
# contraction
tn.contractors.auto(uppernodes + lowernodes + opnode)

L -> R

In [ ]:
svdup = tn.split_node_full_svd(uppernodes[0], left_edges=[phys_edgs[0]], right_edges=[upper_edgs[0]])
svdlow = tn.split_node_full_svd(lowernodes[0], left_edges=[phys_edgs[0]], right_edges=[lower_edgs[0]])
#tn.contractors.auto(uppernodes + lowernodes)

In [ ]:
svdup

R -> L